# 02 — Reporting: plots y export a PDF

In [3]:
#Importación de librerías

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
sns.set_theme(context="notebook", style="whitegrid")
from mcport.providers import YahooProvider
from mcport import PriceSeries, Portfolio, MonteCarloSimulation, MonteCarloPlots, var_cvar

# --- Tickers de la cartera diversificada (USD) ---
symbols = ["AAPL", "NVDA", "XOM", "NEE", "LMT", "CAT"]

# Instancia del proveedor Yahoo
provider = YahooProvider()

# 3 años (~756 días hábiles)
dict_pricehistory = provider.price_history(symbols=symbols, periods=756)

#Creamos el diccionario de PriceSeries
price_series_dict = {}

for sym, df in dict_pricehistory.items():
    ps = PriceSeries.from_dataframe(
        symbol=sym,
        df=df,
        price_col="adj_close",
        currency="USD",
        provider="YahooFinance",
        asset_type="equity"
    )
    price_series_dict[sym] = ps


# --- Seleccionamos los PriceSeries del diccionario ---
positions = [
    price_series_dict["AAPL"],
    price_series_dict["NVDA"],
    price_series_dict["XOM"],
    price_series_dict["NEE"],
    price_series_dict["LMT"],
    price_series_dict["CAT"],
]

# --- Pesos según bloques temáticos ---
weights = [0.20, 0.20, 0.15, 0.15, 0.15, 0.15]

# --- Creamos la cartera ---
portfolio = Portfolio(
    positions=positions,
    weights=weights,
    name="Cartera_Diversificada",
    currency="USD"
)




# ----------------------------
# 1️⃣ Monte Carlo por activo
# ----------------------------

results_assets = {}
n_days = 126       # 6 meses ~ 126 días hábiles
n_sims = 2000
seed = 42

for sym, ps in price_series_dict.items():
    mc = MonteCarloSimulation(ps, days=n_days, n_sims=n_sims, seed=seed)
    res = mc.simulate_and_summarize()
    results_assets[sym] = res

# ----------------------------
# 2️⃣ Monte Carlo del Portfolio
# ----------------------------

mc_port = MonteCarloSimulation(
    portfolio, days=n_days, n_sims=n_sims, capital_inicial=1000.0, seed=seed
)
res_port = mc_port.simulate_and_summarize()
s = res_port["summary"]
